In [2]:
import os
import shutil
import cv2
import math
import random
import numpy as np
import datetime as dt
import tensorflow 
import keras
from collections import deque
import matplotlib.pyplot as plt
plt.style.use("seaborn")
%matplotlib inline
from sklearn.model_selection import train_test_split
from keras.layers import *
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

C:\Users\Ananya\AppData\Local\Temp\ipykernel_21540\612589392.py:12: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn")


In [3]:
from IPython.display import HTML
from base64 import b64encode

# To Show a Video in Notebook
def Play_Video(filepath):
    html = ''
    video = open(filepath,'rb').read()
    src = 'data:video/mp4;base64,' + b64encode(video).decode()
    html += '<video width=640 muted controls autoplay loop><source src="%s" type="video/mp4"></video>' % src 
    return HTML(html)

In [4]:
IMAGE_HEIGHT , IMAGE_WIDTH = 64, 64
SEQUENCE_LENGTH = 16
 
DATASET_DIR = os.path.join('archive','real life violence situations','Real Life Violence Dataset')
 
CLASSES_LIST = ["NonViolence","Violence"]

In [5]:
# Checking

new_model = tf.keras.models.load_model('saved_model/main_bilstm_model')

# Check its architecture
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 16, 2, 2, 1280)   2257984   
 ibuted)                                                         
                                                                 
 dropout (Dropout)           (None, 16, 2, 2, 1280)    0         
                                                                 
 time_distributed_1 (TimeDis  (None, 16, 5120)         0         
 tributed)                                                       
                                                                 
 bidirectional (Bidirectiona  (None, 64)               1319168   
 l)                                                              
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                        

In [6]:
def test_predict_video(video_file_path, SEQUENCE_LENGTH):
 
    video_reader = cv2.VideoCapture(video_file_path)
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frames_list = []
    predicted_class_name = ''
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH),1)
    
    for frame_counter in range(SEQUENCE_LENGTH):
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
        ret, frame = video_reader.read() 
 
        if not ret:
            break
 
    # Prediction
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        normalized_frame = resized_frame / 255
        frames_list.append(normalized_frame)
    predicted_labels_probabilities = new_model.predict(np.expand_dims(frames_list, axis = 0))[0]
    predicted_label = np.argmax(predicted_labels_probabilities)
    predicted_class_name = CLASSES_LIST[predicted_label]
    print(f'Predicted: {predicted_class_name}\nConfidence: {predicted_labels_probabilities[predicted_label]}')
        
    video_reader.release()

In [6]:
input_video_file_path = "archive/real life violence situations/Real Life Violence Dataset/Violence/V_54.mp4"
test_predict_video(input_video_file_path, SEQUENCE_LENGTH)

Play_Video(input_video_file_path)

1/1 [==============================] - 4s 4s/step
Predicted: Violence
Confidence: 0.9988957643508911


In [8]:
# Test Video Capture

cap = cv2.VideoCapture(0)  # Use index 0 for the default camera

duration = 5  # 5 seconds
fps = cap.get(cv2.CAP_PROP_FPS)
num_frames = int(duration * fps)

frames = []  # List to store frames

for _ in range(num_frames):
    # Read a frame from the video capture
    ret, frame = cap.read()

    # Check if the frame has a valid size
    if not ret or frame is None or frame.shape[0] == 0 or frame.shape[1] == 0:
        continue

    frames.append(frame)

    # Display the frame in a window
    cv2.imshow('Resized frame', frame)

    if cv2.waitKey(1) == 27:  # Press 'Esc' to stop capturing
        break

cap.release()
cv2.destroyAllWindows()

output_filename = 'captured_video.mp4'
output_fps = 30  # Frames per second for the output video
output_size = (frames[0].shape[1], frames[0].shape[0])  # Frame size

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_filename, fourcc, output_fps, output_size)

for frame in frames:
    out.write(frame)

out.release()

In [9]:
input_video_file_path = "captured_video.mp4"
test_predict_video(input_video_file_path, SEQUENCE_LENGTH)

Play_Video(input_video_file_path)

1/1 [==============================] - 0s 77ms/step
Predicted: NonViolence
Confidence: 1.0


In [ ]:
while True:
    
    # CAPTURE LIVE FEED
    cap = cv2.VideoCapture(0)  # Use index 0 for the default camera

    duration = 5  # 5 seconds
    fps = cap.get(cv2.CAP_PROP_FPS)
    num_frames = int(duration * fps)

    frames = []  # List to store frames

    for _ in range(num_frames):
        # Read a frame from the video capture
        ret, frame = cap.read()

        # Check if the frame has a valid size
        if not ret or frame is None or frame.shape[0] == 0 or frame.shape[1] == 0:
            continue

        frames.append(frame)

        # Display the frame in a window
        cv2.imshow('Resized frame', frame)

        if cv2.waitKey(1) == 27:  # Press 'Esc' to stop capturing
            break

    cap.release()

    output_filename = 'captured_video.mp4'
    output_fps = 30  # Frames per second for the output video
    output_size = (frames[0].shape[1], frames[0].shape[0])  # Frame size

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_filename, fourcc, output_fps, output_size)

    for frame in frames:
        out.write(frame)

    out.release()
    
    
    # PREDICT DEADFEED
    input_video_file_path = "captured_video.mp4"
    test_predict_video(input_video_file_path, SEQUENCE_LENGTH)

    Play_Video(input_video_file_path)
    
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
    
cv2.destroyAllWindows()


1/1 [==============================] - 2s 2s/step
Predicted: Violence
Confidence: 0.9999998807907104
1/1 [==============================] - 0s 45ms/step
Predicted: Violence
Confidence: 0.9999536275863647
1/1 [==============================] - 0s 63ms/step
Predicted: NonViolence
Confidence: 1.0
1/1 [==============================] - 0s 48ms/step
Predicted: NonViolence
Confidence: 0.9999904632568359
1/1 [==============================] - 0s 47ms/step
Predicted: NonViolence
Confidence: 1.0
1/1 [==============================] - 0s 47ms/step
Predicted: NonViolence
Confidence: 1.0
1/1 [==============================] - 0s 31ms/step
Predicted: NonViolence
Confidence: 1.0
1/1 [==============================] - 0s 48ms/step
Predicted: NonViolence
Confidence: 0.9998080134391785
1/1 [==============================] - 0s 63ms/step
Predicted: NonViolence
Confidence: 0.9999979734420776
1/1 [==============================] - 0s 40ms/step
Predicted: NonViolence
Confidence: 1.0
1/1 [==================